In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import pandas as pd
from os.path import join
from glob import glob
from TorchEASE.src.main.EASE import TorchEASE
import time
import torch
import copy
from tqdm.notebook import tqdm

In [3]:
data_path = 'data/msd/pro_sg'

In [4]:
train_df = pd.read_csv(join(data_path, f"train.csv"))

val_df_tr = pd.read_csv(join(data_path, f"validation_tr.csv"))
val_df_te = pd.read_csv(join(data_path, f"validation_te.csv"))

test_df_tr = pd.read_csv(join(data_path, f"test_tr.csv"))
test_df_te = pd.read_csv(join(data_path, f"test_te.csv"))

In [5]:
train_df.head()

,uid,sid
0,44118,0
1,44118,1
2,44118,2
3,44118,3
4,44118,4


In [7]:
def recall_at_k(actual, predicted, k):
    act_set = set(actual)
    pred_set = set(predicted[:k])

    min_size = np.min([len(act_set), len(pred_set)])
    min_size = len(act_set)

    result = len(act_set & pred_set) / float(min_size)
    return result

In [8]:
def evaluate(ease_model, eval_tr, eval_te, k):
    predictions = ease_model.predict(eval_tr, k)

    uid_to_prediction = dict(zip(eval_tr.uid.unique(), predictions.numpy()))

    preds_with_true = eval_te.groupby("uid").agg({'sid': list})
    preds_with_true = preds_with_true.reset_index()
    preds_with_true['preds'] = preds_with_true.apply(lambda x: uid_to_prediction.get(x.uid), axis=1)

    return preds_with_true.apply(lambda x: recall_at_k(x.sid, x.preds, k), axis=1).mean()

In [9]:
all_results_df = pd.DataFrame(columns=['alpha', 'bagging_size', 'r_20', 'r_50', 'random_state'])

In [ ]:
i_size = train_df.sid.nunique()

ii = 0
for random_state in tqdm(range(0,5)):
    for alpha in tqdm(np.arange(0,0.1,0.03)):
        for bagging_size in tqdm([1,5,10]):
            final_b = torch.zeros((i_size,i_size))

            for i in tqdm(range(bagging_size)):
                print(f'random_state:{random_state} - alpha:{alpha} - bagging_size:{bagging_size} - i:{i}')
                train_df_copy = train_df.copy()
                
                inner_random = random_state * 10 + i
                train_df_copy = train_df_copy.sample(frac = 1-alpha, replace=False, random_state=inner_random)

                te_implicit = TorchEASE(train_df_copy, user_col="uid", item_col="sid")
                te_implicit.fit()
                final_b += te_implicit.B

            final_b /= bagging_size
            te_implicit.B = final_b

            r_20 = evaluate(te_implicit, test_df_tr, test_df_te, k=20)
            r_50 = evaluate(te_implicit, test_df_tr, test_df_te, k=50)
            print(f'r_20:{r_20}, r_50:{r_50}')

            all_results_df.loc[ii] = [alpha, bagging_size, r_20, r_50, random_state]
            all_results_df.to_csv('all_results.csv', index=False)
            ii += 1

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

random_state:0 - alpha:0.0 - bagging_size:1 - i:0
r_20:0.23177472276262198, r_50:0.37100706000462846


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

random_state:4 - alpha:0.03 - bagging_size:1 - i:0
r_20:0.2311602762677328, r_50:0.37078601440198117


  0%|          | 0/5 [00:00<?, ?it/s]

random_state:4 - alpha:0.03 - bagging_size:5 - i:0
random_state:4 - alpha:0.03 - bagging_size:5 - i:1
random_state:4 - alpha:0.03 - bagging_size:5 - i:2
random_state:4 - alpha:0.03 - bagging_size:5 - i:3
random_state:4 - alpha:0.03 - bagging_size:5 - i:4
